In [ ]:
ENABLE_TRANSFORM_TRT_MODEL = True
# ENABLE_TRANSFORM_TRT_MODEL = False

First, create the model. This must match the model used in the interactive training notebook.

In [ ]:
import torch
import torchvision

CATEGORIES = ['apex']
if ENABLE_TRANSFORM_TRT_MODEL:
    device = torch.device('cuda')
    model = torchvision.models.resnet18(pretrained=False)
    model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
    model = model.cuda().eval().half()

Next, load the saved model.  Enter the model path you used to save.

In [ ]:
if ENABLE_TRANSFORM_TRT_MODEL:
    model.load_state_dict(torch.load('road_following_model.pth'))

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [ ]:
from torch2trt import torch2trt

if ENABLE_TRANSFORM_TRT_MODEL:
    
    data = torch.zeros((1, 3, 224, 224)).cuda().half()

    model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [ ]:
if ENABLE_TRANSFORM_TRT_MODEL:
    torch.save(model_trt.state_dict(), 'road_following_model_trt.pth')

Load the optimized model by executing the cell below

In [ ]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model_trt.pth'))

Create the racecar class

In [ ]:
# from jetracer.nvidia_racecar import NvidiaRacecar
# car = NvidiaRacecar()

from Rosmaster_Lib import Rosmaster
# car = Rosmaster(com='/dev/ttyUSB0')
car = Rosmaster()

Create the camera class.

In [ ]:
# from jetcam.csi_camera import CSICamera
# camera = CSICamera(width=224, height=224, capture_fps=65)

from jetcam.usb_camera import USBCamera
camera = USBCamera(width=224, height=224)

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [ ]:
from utils import preprocess
import numpy as np

STEERING_GAIN = 45
STEERING_BIAS = 0.00

speed = 0.3

car.set_beep(100)

try:
    while True:
        image = camera.read()
        image = preprocess(image).half()
        output = model_trt(image).detach().cpu().numpy().flatten()
        x = float(output[0])
        fvalue = x * STEERING_GAIN + STEERING_BIAS
        car.set_akm_steering_angle(fvalue, True)
        # car.set_car_motion(speed, fvalue, 0)
except:
    del camera